In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import seaborn as sns

module_path = (Path().resolve().parent/ "Modules")
sys.path.append(str(module_path))

pd.set_option("display.max_columns", None)

import my_modules, model_tuner, features # 自作モジュール

In [2]:
df = pd.read_csv("../Data/train_data_tmp.csv", encoding="shift-jis")
odds_df = pd.read_csv("../Data/Time_Series_Odds_win_odds.csv", encoding="shift-jis")

In [3]:
df = my_modules.preprocessing(df)
df = my_modules.common_process(df)

C:\Users\ken05\Documents\others\HORSE_RACING\Modules\my_modules.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["place_num"] = df["place"].replace(place_dict).astype(int)


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 252411 entries, 252634 to 0
Data columns (total 74 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   PCI                  251103 non-null  float64       
 1   PCI3                 252411 non-null  float64       
 2   RPCI                 252411 non-null  float64       
 3   race_id              252411 non-null  int64         
 4   year                 252411 non-null  int64         
 5   month                252411 non-null  int64         
 6   day                  252411 non-null  int64         
 7   times                252411 non-null  int64         
 8   place                252411 non-null  object        
 9   daily                252411 non-null  object        
 10  race_num             252411 non-null  int64         
 11  horse                252411 non-null  object        
 12  jockey_id            252411 non-null  object        
 13  trainer_id         

In [7]:
tmp = df[df["id_for_fold"].isin(df["id_for_fold"].unique().tolist()[:10])]
tmp 

,PCI,PCI3,RPCI,race_id,year,month,day,times,place,daily,race_num,horse,jockey_id,trainer_id,horse_N,waku_num,horse_num,class_code,track_code,corner_num,dist,state,weather,age_code,sex,age,basis_weight,blinker,weight,inc_dec,weight_code,win_odds,win_odds_1,win_odds_1_pop,win_odds_2,win_odds_2_pop,win_mul_odds_Hi,win_mul_odds_Lo,win_mul_odds_1_Hi,win_mul_odds_1_Lo,win_mul_odds_1_pop,win_mul_odds_2_Hi,win_mul_odds_2_Lo,win_mul_odds_2_pop,rank,time_diff,time,corner1_rank,corner2_rank,corner3_rank,corner4_rank,last_3F_time,last_3F_rank,Ave_3F,last_3F_time_diff,leg,pop,prize,error_code,father,mother,broodmare_sire,broodmare_sire_type,horse_color,id,id_for_fold,field_type,flat_or_jump,turn_type,race_type,waku,datetime,target,target3
252634,38.7,40.57,36.0,202001050601010115,2020,1,5,1,中山,1,1,ラブカワールド,1177,1131,16,8,15,7,24,2,1200,良,晴,12,牝,3,51.0,NaN,436.0,4.0,3,474.2,355.9,15,254.1,14,153.0,39.3,115.2,36.0,15,74.6,30.0,14,13,2.0,1153,NaN,NaN,15.0,16.0,39.9,9,35.40,1.5,後方,15.0,0,0,トゥザワールド,ラブカテリーナ,オレハマッテルゼ,ロイヤルチャージャー系,黒鹿,2017104350,2020010506010101,ダート,平地,R,中山ダート1200,outer,2020-01-05 01:02:00,0,0
252635,36.6,40.57,36.0,202001050601010116,2020,1,5,1,中山,1,1,シラカワカツコ,1173,1031,16,8,16,7,24,2,1200,良,晴,12,牝,3,51.0,NaN,404.0,4.0,3,233.1,239.5,13,197.1,13,83.6,21.6,62.3,19.6,13,30.6,12.4,12,14,2.1,1154,NaN,NaN,9.0,10.0,40.4,13,35.00,1.1,中団,13.0,0,0,ディープブリランテ,シャイニングピサ,Smart Strike,ネイティヴダンサー系,鹿毛,2017101747,2020010506010101,ダート,平地,R,中山ダート1200,outer,2020-01-05 01:02:00,0,0
252633,40.6,40.57,36.0,202001050601010102,2020,1,5,1,中山,1,1,スルーザリミッツ,1122,1118,16,1,2,7,24,2,1200,良,晴,12,牝,3,54.0,NaN,424.0,-4.0,3,31.4,31.1,7,22.6,6,13.7,3.9,11.3,3.8,7,7.4,3.2,6,12,1.6,1149,NaN,NaN,16.0,15.0,39.3,6,35.60,1.7,後方,7.0,0,0,パイロ,ファービヨンド,ディープスカイ,ロイヤルチャージャー系,黒鹿,2017101861,2020010506010101,ダート,平地,R,中山ダート1200,inner,2020-01-05 01:02:00,0,0
252632,36.3,40.57,36.0,202001050601010104,2020,1,5,1,中山,1,1,セルレア,1161,1115,16,2,4,7,24,2,1200,良,晴,12,牝,3,53.0,NaN,464.0,10.0,3,14.5,15.7,4,19.5,5,7.3,2.2,6.2,2.2,5,6.1,2.7,5,11,1.6,1149,NaN,NaN,6.0,6.0,40.2,11,34.70,0.8,中団,4.0,0,0,ロードカナロア,ダイワミストレス,ダイワメジャー,ロイヤルチャージャー系,鹿毛,2017102095,2020010506010101,ダート,平地,R,中山ダート1200,inner,2020-01-05 01:02:00,0,0
252631,32.9,40.57,36.0,202001050601010109,2020,1,5,1,中山,1,1,サノレーヌ,1164,1141,16,5,9,7,24,2,1200,良,晴,12,牝,3,51.0,NaN,438.0,0.0,3,9.5,8.0,3,6.8,3,4.5,1.5,3.9,1.5,3,3.1,1.5,3,10,1.5,1148,NaN,NaN,1.0,1.0,40.9,15,33.90,0.0,逃げ,3.0,0,0,サウスヴィグラス,アニマートホウヨウ,フサイチコンコルド,ニアークティック系,鹿毛,2017100847,2020010506010101,ダート,平地,R,中山ダート1200,outer,2020-01-05 01:02:00,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252746,45.0,51.23,45.8,202001050801010515,2020,1,5,1,京都,1,5,バトーデュシエル,1088,1164,16,8,15,7,17,2,1600,良,曇,12,牝,3,54.0,NaN,468.0,-4.0,3,5.3,5.1,2,4.6,1,2.2,1.7,2.4,1.8,2,2.3,1.8,2,4,0.8,1374,NaN,NaN,3.0,1.0,37.7,9,35.82,0.3,先行,3.0,77,0,ロードカナロア,エルダンジュ,サンデーサイレンス,ロイヤルチャージャー系,鹿毛,2017105252,2020010508010105,芝,平地,R,京都芝1600,outer,2020-01-05 05:05:00,0,0
252745,52.6,51.23,45.8,202001050801010510,2020,1,5,1,京都,1,5,ナリタブレード,5203,1069,16,5,10,7,17,2,1600,良,曇,12,牡,3,56.0,NaN,506.0,16.0,3,46.1,33.6,11,30.4,11,16.6,11.2,13.1,9.1,12,10.1,7.4,12,3,0.7,1373,NaN,NaN,16.0,14.0,35.9,2,36.84,2.0,後方,11.0,130,0,ヴィクトワールピサ,レディトゥプリーズ,More Than Ready,ロイヤルチャージャー系,黒鹿,2017104988,2020010508010105,芝,平地,R,京都芝1600,outer,2020-01-05 05:05:00,0,1
252744,48.7,51.23,45.8,202001050801010501,2020,1,5,1,京都,1,5,ヒルノエドワード,1015,1044,16,1,1,7,17,2,1600,良,曇,12,牡,3,56.0,NaN,476.0,-2.0,3,9.2,11.7,7,12.3,6,3.5,2.4,3.5,2.5,5,3.4,2.6,4,2,0.2,1368,NaN,NaN,8.0,6.0,36.6,3,36.12,0.8,中団,6.0,200,0,ルーラーシップ,チャオプラヤ,ネオユニヴァース,ロイヤルチャージャー系,黒鹿,2017101322,2020010508010105,芝,平地,R,京都芝1600,inner,2020-01-05 05:05:00,0,1
252743,52.4,51.23,45.8,202001050801010505,2020,1,5,1,京都,1,5,エアファンディタ,1093,1144,16,3,5,7,17,2,1600,良,曇,12,牡,3,56.0,NaN,

### 後で実験をし直す予定